# Water Values for Long-Duration Storage Operation

This example demonstrates how the concept of **water values** (known from hydroelectric power plants) can be used to capture seasonal behaviour of long-duration storage in a rolling horizon optimisation with limited operational foresight. Applying the concept of water values helps internalise the future value of stored energy beyond the optimisation horizon when making short-term operational decisions.

In [ ]:
import logging

import matplotlib.pyplot as plt
import pandas as pd

import pypsa

pypsa.options.params.optimize.solver_name = "gurobi"
logging.getLogger("gurobipy").setLevel(logging.ERROR)

For this example, we consider a simple energy system with a single bus and 3-hourly time steps over one year (8760 hours). We have time-varying load, wind and solar generation at zero marginal cost, a gas generator with marginal cost function $C'(g) = 80 + 0.01 g$ €/MWh, and hydrogen storage consisting of electrolysis, turbine, and a non-cyclic storage tank with a set initial energy. All components have fixed capacities; investments are not optimised here. 

In [ ]:
n = pypsa.examples.model_energy()

n.remove("Generator", "load shedding")
n.remove("StorageUnit", "battery storage")

n.generators.loc[["solar", "wind"], "p_nom"] = 30_000, 20_000
n.generators.p_nom_extendable = False

n.links.loc[["electrolysis", "turbine"], "p_nom"] = 10_000, 20_000
n.links.p_nom_extendable = False

n.stores.loc["hydrogen storage", ["e_nom", "e_initial"]] = 2_000_000, 500_000
n.stores.e_nom_extendable = False
n.stores.e_cyclic = False

n.add("Carrier", "gas", color="darkorange")
n.add(
    "Generator",
    "gas",
    bus="electricity",
    p_nom=40_000,
    marginal_cost=80,
    marginal_cost_quadratic=0.01,
    carrier="gas",
)

## Case 1: Perfect Foresight

Let's first solve the problem with perfect foresight over the whole year to obtain a benchmark solution that captures the optimal seasonal operation of the hydrogen storage.

In [ ]:
n.optimize(assign_all_duals=True, log_to_console=False)

Here, we get fuel costs of 1160 M€/a.

In [ ]:
obj = n.statistics.opex(components="Generator").sum() / 1e6
obj

Electricity prices tend to be slightly higher in the winter months.

In [ ]:
n.buses_t.marginal_price["electricity"].resample("D").mean().plot(
    figsize=(6, 2), ylabel="€/MWh", xlabel=""
);

In [ ]:
pdc = (
    n.buses_t.marginal_price["electricity"]
    .sort_values(ascending=False)
    .reset_index(drop=True)
)
pdc.plot(figsize=(5, 3), ylabel="€/MWh", xlabel="snapshots", xlim=(0, 8760 / 3));

The price at the hydrogen bus reflects the marginal storage value (MSV) of hydrogen. The MSV indicates the willingness to pay for an additional unit of hydrogen stored at that time and, thereby, reflects its future value. This is the equivalent of water values in hydro storage.

In [ ]:
msv = n.buses_t.marginal_price["hydrogen"]
msv.plot(figsize=(6, 2), ylim=(0, 100), ylabel="€/MWh", xlabel="");

As in this case with perfect foresight, the model foresees future scarcity periods in electricity supply, the storage is operated seasonally, retaining high storage levels over the summer to cover scarcity periods in winter. 

In [ ]:
soc = n.stores_t.e.div(1e3).squeeze()
soc.plot(figsize=(6, 2), ylabel="GWh", xlabel="", legend=False);

## Case 2: Myopic Foresight

Now, we solve the same dispatch problem, but with myopic foresight of 15 days (120 snapshots at 3-hourly resolution). This means that at each optimisation step, the model only considers the next 15 days when making operational decisions, without knowledge of future periods beyond this horizon.

In [ ]:
n.model.solver_model = None
n2 = n.copy()

n2.optimize.optimize_with_rolling_horizon(
    assign_all_duals=True,
    horizon=120,
    overlap=0,
    log_to_console=False,
)

This leads to higher fuel costs of 1337 M€/a, as the storage is not optimally utilised for seasonal shifting and the gas generator must step in.

In [ ]:
obj2 = n2.statistics.opex(components="Generator").sum() / 1e6
obj2

The price duration curve now has more periods with higher prices where the gas generator is dispatched, and more periods with lower prices where renewable generation may be curtailed and the value of storage is not seen.

In [ ]:
pdc2 = (
    n2.buses_t.marginal_price["electricity"]
    .sort_values(ascending=False)
    .reset_index(drop=True)
)
pdc2.plot(figsize=(5, 3), ylabel="€/MWh", xlabel="snapshots", xlim=(0, 8760 / 3));

The consequence of myopic operation is that the storage has no incentive to retain energy for future scarcity periods beyond the 15-day horizon. This leads to the absence of a seasonal storage pattern.

In [ ]:
soc2 = n2.stores_t.e.div(1e3).squeeze()
soc2.plot(figsize=(6, 2), ylabel="GWh", xlabel="", legend=False);

## Case 3: Myopic Foresight with Water Values

Now, we amend the myopic foresight dispatch optimisation over 15 days at a time by providing information about the marginal storage value from the perfect foresight case.

We do this by setting the `marginal_cost` attribute of the hydrogen `Store` component to the MSV time series we obtained previously. In this way, we tell the model for each time step at what price the hydrogen storage should buy or sell hydrogen. From the angle of the electricity bus, the hydrogen storage would be willing to buy/charge when the electricity price is below the MSV (multiplied by the electrolysis efficiency) and sell/discharge when the electricity price is above the MSV (divided by the turbine efficiency).

In [ ]:
n3 = n.copy()

n3.stores_t.marginal_cost["hydrogen storage"] = msv

n3.optimize.optimize_with_rolling_horizon(
    assign_all_duals=True,
    horizon=120,
    overlap=0,
    log_to_console=False,
)

When we take exactly the same MSV time series from the perfect foresight case, we recover the **exactly** same optimal seasonal operation, price duration curve and fuel costs of 1160 M€/a. From a theoretical perspective, this applies the concept of Lagrangian relaxation, where the MSV act as shadow prices that internalise the inter-temporal coupling of the optimal storage operation across the full year, even when optimising only over a limited horizon at a time.

In [ ]:
obj3 = n3.statistics.opex(components="Generator").sum() / 1e6
obj3

In [ ]:
pdc3 = (
    n3.buses_t.marginal_price["electricity"]
    .sort_values(ascending=False)
    .reset_index(drop=True)
)
pdc3.plot(figsize=(5, 3), ylabel="€/MWh", xlabel="snapshots", xlim=(0, 8760 / 3));

In [ ]:
soc3 = n3.stores_t.e.div(1e3).squeeze()
soc3.plot(figsize=(6, 2), ylabel="GWh", xlabel="", legend=False);

## Case 4: Myopic Foresight with Approximate Water Values

In practice, we may not have access to the exact MSV time series from a perfect foresight case. Instead, we can use approximate water values derived from historical data, forecasts, or simplified models. In the example below, we just use the annual average MSV as a constant value over the year, which works reasonably well for a storage medium with modest variation in water values over time.

In [ ]:
n4 = n.copy()
n4.stores_t.marginal_cost["hydrogen storage"] = msv.mean()

n4.optimize.optimize_with_rolling_horizon(
    assign_all_duals=True,
    horizon=120,
    overlap=0,
    log_to_console=False,
)

Now, we do not exactly reproduce the perfect foresight solution, but still do not deviate too far from it. The fuel costs are 1179 M€/a, and the general seasonal storage operation is retained, albeit with some deviations compared to the perfect foresight case.

In [ ]:
obj4 = n4.statistics.opex(components="Generator").sum() / 1e6
obj4

In [ ]:
pdc4 = (
    n4.buses_t.marginal_price["electricity"]
    .sort_values(ascending=False)
    .reset_index(drop=True)
)

In [ ]:
soc4 = n4.stores_t.e.div(1e3).squeeze()
soc4.plot(figsize=(6, 2), ylabel="GWh", xlabel="", legend=False);

## Summary

**The cases above demonstrate that even approximate water values can significantly improve the operation of long-duration storage under myopic foresight.**

The tables and figures below summarise the differences between the four cases in terms of fuel costs, price duration curves, and storage operation.

**Fuel costs:**

In [ ]:
pd.Series(
    {
        "perfect foresight": obj,
        "rolling horizon": obj2,
        "rolling horizon + water values": obj3,
        "rolling horizon + avg. water values": obj4,
    }
).round(3)

**Price duration curves:**

In [ ]:
ax = pd.concat(
    {
        "perfect foresight": pdc,
        "rolling horizon": pdc2,
        "rolling horizon + water values": pdc3,
        "rolling horizon + avg. water values": pdc4,
    },
    axis=1,
).plot(figsize=(5, 3), ylabel="€/MWh", xlabel="snapshots", xlim=(0, 8760 / 3))
for line in ax.lines[-3:]:
    line.set_linestyle(":")

**Storage operation:**

In [ ]:
ax = pd.concat(
    {
        "perfect foresight": soc,
        "rolling horizon": soc2,
        "rolling horizon + water values": soc3,
        "rolling horizon + avg. water values": soc4,
    },
    axis=1,
).plot(figsize=(6, 2), ylabel="GWh", xlabel="")
for line in ax.lines[-3:]:
    line.set_linestyle(":")

plt.legend(bbox_to_anchor=(1.02, 1), loc="upper left")
